# SFHo/SFHx example for O$_2$sclpy

See the O$_2$sclpy documentation at
https://neutronstars.utk.edu/code/o2sclpy for more information.

In [1]:
import o2sclpy
import matplotlib.pyplot as plot
import numpy
import sys

plots=True
if 'pytest' in sys.modules:
    plots=False

Link the O$_2$scl library:

In [2]:
link=o2sclpy.linker()
link.link_o2scl()

Get the value of $\hbar c$ from an O$_2$scl find_constants object:

In [3]:
fc=o2sclpy.find_constants(link)
hc=fc.find_unique('hbarc','MeV*fm')
print('hbarc = %7.6e' % (hc))

hbarc = 1.973270e+02


Get a copy (a pointer to) the O$_2$scl unit conversion object:

In [4]:
cu=link.o2scl_settings.get_convert_units()

In [5]:
sfho=o2sclpy.eos_had_rmf(link)
o2sclpy.rmf_load(link,sfho,'SFHo')
sfhx=o2sclpy.eos_had_rmf(link)
o2sclpy.rmf_load(link,sfhx,'SFHx')

Compute nuclear saturation and output the saturation density
and binding energy:

In [6]:
sfho.saturation()
print(('SFHo: n0=%7.6e 1/fm^3, E/A=%7.6e MeV, K=%7.6e MeV, '+
       'M*/M=%7.6e, S=%7.6e MeV, L=%7.6e MeV') % 
      (sfho.n0,sfho.eoa*hc,sfho.comp*hc,sfho.msom,sfho.esym*hc,
       sfho.fesym_slope(sfho.n0)*hc))
print('')

SFHo: n0=1.582415e-01 1/fm^3, E/A=-1.617240e+01 MeV, K=2.452197e+02 MeV, M*/M=7.610394e-01, S=3.123132e+01 MeV, L=4.574132e+01 MeV



In [7]:
sfhx.saturation()
print(('SFHx: n0=%7.6e 1/fm^3, E/A=%7.6e MeV, K=%7.6e MeV, '+
       'M*/M=%7.6e, S=%7.6e MeV, L=%7.6e MeV') % 
      (sfhx.n0,sfhx.eoa*hc,sfhx.comp*hc,sfhx.msom,sfhx.esym*hc,
       sfhx.fesym_slope(sfhx.n0)*hc))
print('')

SFHx: n0=1.600292e-01 1/fm^3, E/A=-1.614755e+01 MeV, K=2.386343e+02 MeV, M*/M=7.181083e-01, S=2.827242e+01 MeV, L=2.318892e+01 MeV



Baryon density grid in $1/\mathrm{fm}^3$. The O$_2$scl object
`uniform_grid_end_width` is like numpy.arange() but is numerically
stable.

In [ ]:
ug_nb=o2sclpy.uniform_grid_end_width.init(link,0.01,1.0,0.01)

Temperature grid in MeV

In [ ]:
ug_T=o2sclpy.uniform_grid_end_width.init(link,0.1,10.0,0.1)

Store the EOS in a table3d object

In [ ]:
t3d=o2sclpy.table3d(link)
t3d.set_xy_grid('nB',ug_nb,'T',ug_T)

Create a new slice for the energy per baryon

In [ ]:
t3d.new_slice('EoA')

In [ ]:
n=sfho.get_def_neutron()
p=sfho.get_def_proton()
th=sfho.get_def_thermo()

In [ ]:
for i in range(0,t3d.get_nx()):
   for j in range(0,t3d.get_ny()):
       n.n=ug_nb[i]/2.0
       p.n=ug_nb[i]/2.0
       sfho.calc_temp_e(n,p,ug_T[j]/197.33,th)
       t3d.set(i,j,'EoA',th.ed/ug_nb[i])

In [ ]:
if plots:
    plot.den_plot(t3d,'EoA')
    plot.show()

In [8]:
#xarr=[i*0.02+0.02 for i in range(0,16)]
#for T in numpy.arange(0,20,5):
#   sfho.calc_

In [9]:
def test_fun():
    assert numpy.allclose(sfho.n0,0.1582415,rtol=1.0e-4)
    assert numpy.allclose(sfhx.n0,0.1600292,rtol=1.0e-4)
    return